In [13]:
from flask import Flask, jsonify
from mysql.connector import Error
import mysql.connector
import pandas as pd
import os

##### DB 연결

In [3]:
# MySQL 데이터베이스 연결 설정
db_config = {
    "host": "localhost",
    "user": "root",  # 본인의 MySQL 사용자 이름으로 변경
    "password": "wjdtkfkd12",  # 본인의 MySQL 비밀번호로 변경
    "database": "jiggle",
    "port": 3306,
    "auth_plugin": "mysql_native_password"
}

In [10]:
class DB_CONNECTOR:
    def __init__(self, db_config):
        self.db_config = db_config

    def create_table(self, sql):
        try:
            # 데이터베이스 연결 생성
            conn = mysql.connector.connect(**db_config)

            if conn.is_connected():
                cursor = conn.cursor()

                print("connected")
                
                # SQL 쿼리 실행
                cursor.execute(sql)
            
                print("Table Created")

            # 조회 완료했으니 DB 연결 해제
            if conn.is_connected():
                cursor.close()
                conn.close()

        except Error as e:
            print(f"오류 : {e}")
    
    def upload_data(self, sql, df):
        df = df.iloc[:, 1:]
        df = df.where(pd.notnull(df), "no data")
        
        try:
            # 데이터베이스 연결 생성
            conn = mysql.connector.connect(**db_config)

            if conn.is_connected():
                cursor = conn.cursor()

                print("connected")
                
                # SQL 쿼리 실행
                for _, row in df.iterrows():
                    cursor.execute(sql, tuple(row))
                conn.commit()
                print("Upload Complete..!")

            # 조회 완료했으니 DB 연결 해제
            if conn.is_connected():
                cursor.close()
                conn.close()

        except Error as e:
            print(f"오류 : {e}")

    def read_data(self, sql):
        try:
            # 데이터베이스 연결 생성
            conn = mysql.connector.connect(**db_config)

            if conn.is_connected():
                cursor = conn.cursor()

                print("connected")
                
                # SQL 쿼리 실행
                cursor.execute(sql)
            
                result = cursor.fetchall()  # 실행한 결과
                columns = [desc[0] for desc in cursor.description]  # 컬럼 이름 가져오기
                result_df = pd.DataFrame(result, columns=columns)

            # 조회 완료했으니 DB 연결 해제
            if conn.is_connected():
                cursor.close()
                conn.close()

        except Error as e:
            print(f"오류 : {e}")

        return result_df

##### 데이터 불러오기

In [9]:
df = pd.read_csv('./public/data/articles_pre.csv')

##### 데이터 업로드하기

In [11]:
# 테이블 생성
create_table_sql = """CREATE TABLE IF NOT EXISTS Articles (
    id INT AUTO_INCREMENT PRIMARY KEY,
    placeName VARCHAR(25),
    address VARCHAR(50),
    link VARCHAR(100),
    comment VARCHAR(50),
    rate INT,
    platform VARCHAR(20)
)"""

# 데이터 업로드
data_upload_sql = """
    INSERT INTO Articles (placeName, address, link, comment, rate, platform)
    VALUES (%s, %s, %s, %s, %s, %s)
"""
    
# 테이블 확인
show_data_sql = "SELECT * FROM articles;"

# db연결
myConnector = DB_CONNECTOR(db_config)
#myConnector.create_table(create_table_sql)
#myConnector.upload_data(data_upload_sql, df)
new_df = myConnector.read_data(show_data_sql)

connected


In [12]:
new_df

,id,placeName,address,link,comment,rate,platform
0,13,대충유원지,서울 마포구 월드컵북로6길 37 1층,https://brunch.co.kr/@archive-zeze/6,같은 곳 || 건축을 중심으로,3,브런치
1,14,대충유원지,서울 마포구 월드컵북로6길 37 1층,https://brunch.co.kr/@itrv/24,같은 곳 || 디자인을 중심으로,4,브런치
2,15,대충유원지,서울 마포구 월드컵북로6길 37 1층,https://m.sedaily.com/NewsView/1RY5VQGJQN/GK02...,같은 곳 || 기사,2,서울경제
3,16,스페이스깨,서울특별시 종로구 자하문로5길 15 스페이스깨 1층,https://fhhhfriends.com/%EC%8A%A4%ED%8E%98%EC%...,건축 스튜디오에서 작성,2,푸하하프렌즈
4,17,하우스서울,서울 송파구 백제고분로9길 5 1층,https://brunch.co.kr/@ratm820309n85i/292,no data,2,브런치
5,18,주신당,서울 중구 퇴계로 411 1층 6호,https://brunch.co.kr/@itrv/1,"신당동에, 신당. 매우 흥미로움",4,브런치
6,19,아임뮤트,서울 강남구 논현로157길 36 1층,https://brunch.co.kr/@itrv/22,no data,3,브런치
7,20,책바,서울 마포구 포은로 90 3층 301호,https://brunch.co.kr/@itrv/18,no data,3,브런치
8,21,콩크 자재라이브러리,서울 마포구 양화로6길 50 3층,https://brunch.co.kr/@itrv/17,식당/카페가 아닌 편집숍 같은 공간,3,브런치
9,22,묘사서울,서울 성동구 서울숲2길 2 2층,https://brunch.co.kr/@itrv/16,"""한국적인"", 디자인을 중심으로",4,브런치
